# Model prototyping

This notebook trains and publishes prototype models.

In [ ]:
# Add project root to path.
import sys
sys.path.append('../../../../..')

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets.widgets import IntSlider, interact
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, \
    Dropout
from mlops.dataset.versioned_dataset import VersionedDataset
from mlops.examples.image.classification.train_model import train_model, \
    publish_model
from mlops.examples.image.classification.pokemon_classification_data_processor \
    import PokemonClassificationDataProcessor

In [ ]:
DATASET_PUBLICATION_PATH_LOCAL = '../../../../../datasets/pokemon'
DATASET_VERSION = 'v1'
MODEL_PUBLICATION_PATH_LOCAL = '../../../../../models/pokemon'

## Retrieve versioned dataset

In [ ]:
dataset = VersionedDataset(os.path.join(DATASET_PUBLICATION_PATH_LOCAL,
                                        DATASET_VERSION))

## Define prototype model

In [ ]:
model = Sequential()
# Shape: (None, 120, 120, 3).
model.add(Conv2D(16, (3, 3), activation='relu',
                 input_shape=dataset.X_train.shape[1:]))
# Shape: (None, 118, 118, 16).
model.add(MaxPooling2D((2, 2)))
# Shape: (None, 59, 59, 16).
model.add(Conv2D(32, (3, 3), activation='relu'))
# Shape: (None, 57, 57, 32).
model.add(MaxPooling2D((2, 2)))
# Shape: (None, 28, 28, 32).
model.add(Conv2D(64, (3, 3), activation='relu'))
# Shape: (None, 26, 26, 64).
model.add(MaxPooling2D((2, 2)))
# Shape: (None, 13, 13, 64).
model.add(Conv2D(128, (3, 3), activation='relu'))
# Shape: (None, 11, 11, 128).
model.add(MaxPooling2D((2, 2)))
# Shape: (None, 5, 5, 128).
model.add(Conv2D(256, (3, 3), activation='relu'))
# Shape: (None, 3, 3, 256).
model.add(Flatten())
# Shape: (None, 2304).
model.add(Dense(128, activation='relu'))
# Shape: (None, 128).
model.add(Dropout(0.4))
# Shape: (None, 128).
model.add(Dense(dataset.y_train.shape[1], activation='sigmoid'))
# Shape: (None, 18).
model.compile(optimizer='adam', loss='binary_crossentropy')

## Train and publish the versioned model

In [ ]:
training_config = train_model(model,
                              dataset,
                              epochs=5,
                              batch_size=4)
publish_model(model,
              dataset,
              training_config,
              MODEL_PUBLICATION_PATH_LOCAL,
              tags=['prototype'])

## Examine results

In [ ]:
pred = PokemonClassificationDataProcessor.get_valid_prediction(
    model.predict(dataset.X_val))
pred = dataset.data_processor.unpreprocess_labels(pred)
labels = dataset.data_processor.unpreprocess_labels(dataset.y_val)

In [ ]:
def show_img_and_pred(idx: int) -> None:
    """Plots the image at the given index and prints prediction info."""
    print(f'Validation image: {idx}')
    print(f'Label: {labels[idx]}')
    print(f'Prediction: {pred[idx]}')
    fig = plt.imshow(dataset.X_val[idx])
    plt.show()

_ = interact(show_img_and_pred, idx=IntSlider(value=0,
                                              min=0,
                                              max=len(dataset.X_val) - 1,
                                              step=1))